In [1]:
from kafka import KafkaConsumer
import time
import json
from pymongo import MongoClient
import pandas as pd
import os
import csv
client = MongoClient('mongodb://localhost:27017/')
db = client['IctDatabse']
collection = db['Vcb']

ModuleNotFoundError: No module named 'kafka.vendor.six.moves'

In [ ]:
import csv
import pymongo

# Tạo kết nối đến MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['IctDatabse']
collection = db['Vcb']

# Đọc dữ liệu từ tệp CSV và chuyển đổi thành từ điển
def read_csv_to_dict(filename):
    data_list = []
    with open(filename, mode='r') as file:  # Sửa dòng này
        reader = csv.DictReader(file)
        for row in reader:
            data_list.append(row)
    return data_list

# Chèn dữ liệu vào MongoDB
def insert_data_to_mongo(data_list):
    for i in data_list:  # Kiểm tra xem danh sách dữ liệu có rỗng không
        collection.insert_one(i)
        print("ok")

# Đọc dữ liệu từ tệp CSV và chèn vào MongoDB
csv_file = "file_test.csv"
data_list = read_csv_to_dict(csv_file)
insert_data_to_mongo(data_list)


In [ ]:
# import csv
# import pymongo
# import time
# import os

# # Tạo kết nối đến MongoDB
# client = pymongo.MongoClient("mongodb://localhost:27017/")
# db = client['tp4']
# collection = db['t_4']

# # Đọc dữ liệu từ tệp CSV và chuyển đổi thành từ điển
# def read_csv_to_dict(filename):
#     data_list = []
#     with open(filename, mode='r') as file:
#         reader = csv.DictReader(file)
#         for row in reader:
#             data_list.append(row)
#     return data_list

# # Chèn dữ liệu vào MongoDB
# def insert_data_to_mongo(data_list):
#     for i in data_list:
#         print("Data to insert:", i)  # Thêm dòng này để kiểm tra dữ liệu
#         collection.insert_one(i)
#         print("Inserted:", i)

# # Hàm theo dõi sự thay đổi của tệp CSV
# def watch_file(filename, interval=10):
#     last_mtime = None
#     while True:
#         try:
#             current_mtime = os.path.getmtime(filename)
#             print(f"Current mtime: {current_mtime}, Last mtime: {last_mtime}")  # In thông tin mtime
#             if last_mtime is None:
#                 last_mtime = current_mtime

#             if current_mtime > last_mtime:
#                 print(f"Detected change in {filename}. Reading new data...")
#                 data_list = read_csv_to_dict(filename)
#                 insert_data_to_mongo(data_list)
#                 last_mtime = current_mtime
#             time.sleep(interval)
#         except KeyboardInterrupt:
#             print("Stopping file watch.")
#             break

# # Theo dõi tệp CSV và đẩy dữ liệu mới lên MongoDB
# csv_file = "file_test.csv"
# watch_file(csv_file, interval=0.001)


In [ ]:
import csv
import pymongo
import time
import os

# Tạo kết nối đến MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['IctDatabse']
collection = db['Vcb']

# Đọc dữ liệu từ tệp CSV và chuyển đổi thành từ điển
def read_csv_to_dict(filename):
    data_list = []
    with open(filename, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            data_list.append(row)
    return data_list

# Chèn dữ liệu vào MongoDB, chỉ chèn dữ liệu mới
def insert_data_to_mongo(data_list):
    for i in data_list:
        # Kiểm tra xem bản ghi đã tồn tại trong MongoDB chưa
        if not collection.find_one({'Date': i['Date']}):
            print("Inserting new data:", i)
            collection.insert_one(i)
        else:
            print("Data already exists, skipping:", i)

# Hàm theo dõi sự thay đổi của tệp CSV
def watch_file(filename, interval=10):
    last_mtime = None
    while True:
        try:
            current_mtime = os.path.getmtime(filename)
            print(f"Current mtime: {current_mtime}, Last mtime: {last_mtime}")  # In thông tin mtime
            if last_mtime is None:
                last_mtime = current_mtime

            if current_mtime > last_mtime:
                print(f"Detected change in {filename}. Reading new data...")
                data_list = read_csv_to_dict(filename)
                insert_data_to_mongo(data_list)
                last_mtime = current_mtime
            time.sleep(interval)
        except KeyboardInterrupt:
            print("Stopping file watch.")
            break

# Theo dõi tệp CSV và đẩy dữ liệu mới lên MongoDB
csv_file = "file_test.csv"
watch_file(csv_file, interval=1)
